In [9]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, concat

In [2]:
spark = SparkSession.builder.appName('lab200').getOrCreate()

In [3]:
df = spark.read.csv('Restaurants_in_Wake_County_NC.csv',inferSchema=True,header=True)

In [5]:
df.show(5)

+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|OBJECTID|    HSISID|                NAME|            ADDRESS1|ADDRESS2|       CITY|STATE|POSTALCODE|   PHONENUMBER| RESTAURANTOPENDATE|     FACILITYTYPE|PERMITID|           X|          Y|GEOCODESTATUS|
+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|    1001|4092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 05:30:00|       Restaurant|    6952|-78.66818477|35.78783803|            M|
|    1002|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 05:30:00|       Food Stand|    6953|-78.78211173|35.73717591|

In [7]:
df.printSchema()

root
 |-- OBJECTID: integer (nullable = true)
 |-- HSISID: long (nullable = true)
 |-- NAME: string (nullable = true)
 |-- ADDRESS1: string (nullable = true)
 |-- ADDRESS2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- PHONENUMBER: string (nullable = true)
 |-- RESTAURANTOPENDATE: timestamp (nullable = true)
 |-- FACILITYTYPE: string (nullable = true)
 |-- PERMITID: integer (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- GEOCODESTATUS: string (nullable = true)



In [18]:
df = df.withColumn('county',lit('Wake')) \
        .withColumnRenamed('datasetiId','datasetId') \
        .withColumnRenamed('NAME','name') \
        .withColumnRenamed('ADDRESS1','address1') \
        .withColumnRenamed('ADDRESS2','address2') \
        .withColumnRenamed('CITY','city') \
        .withColumnRenamed('STATE','state') \
        .withColumnRenamed('POSTALCODE','zip') \
        .withColumnRenamed('PHONENUMBER','tel') \
        .withColumnRenamed('RESTAURANTOPENDATE','dataStart') \
        .withColumnRenamed('FACILITYTYPE','type') \
        .withColumnRenamed('X','geoX') \
        .withColumnRenamed('Y','geoY') \
        .drop('OBJECTID','PERMITID','GEOCODESTATUS')

In [19]:
df.printSchema()

root
 |-- datasetId: long (nullable = true)
 |-- name: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- dataStart: timestamp (nullable = true)
 |-- type: string (nullable = true)
 |-- geoX: double (nullable = true)
 |-- geoY: double (nullable = true)
 |-- county: string (nullable = false)



In [14]:
df.show(5)

+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+
|datasetiId|                name|            address1|address2|       city|state|       zip|           tel|          dataStart|             type|        geoX|       geoY|county|
+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+
|4092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 05:30:00|       Restaurant|-78.66818477|35.78783803|  Wake|
|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 05:30:00|       Food Stand|-78.78211173|35.73717591|  Wake|
|4092017012|CAROLINA SUSHI &a...|5951-107 POYNER V...|    null|    RALEIGH|   NC|     27616|(919) 981-5835|201

In [22]:
df = df.withColumn('id',concat(col('state'),lit('_'),
                         col('county'),lit('_'),
                         col('datasetId')))

In [23]:
df.show(5)

+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+------------------+
| datasetId|                name|            address1|address2|       city|state|       zip|           tel|          dataStart|             type|        geoX|       geoY|county|                id|
+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+------------------+
|4092016024|                WABA|2502 1/2 HILLSBOR...|    null|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 05:30:00|       Restaurant|-78.66818477|35.78783803|  Wake|NC_Wake_4092016024|
|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    null|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 05:30:00|       Food Stand|-78.78211173|35.73717591|  Wake|NC_Wake_4092021693|
|4092017012|CAR

In [27]:
df.rdd.getNumPartitions()

4

In [26]:
df = df.repartition(4)